In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import requests
import json
from census import Census

In [ ]:
combined_fastfood= pd.read_csv('resources/clean_all_combined.csv')


In [ ]:
#Anne Work - most popular fast food brands by state

In [ ]:
#Julia work - fast food quantity by obesity by state

In [ ]:
#Nicole Work - fast food per capita